<a href="https://colab.research.google.com/github/Aniket1492/PdfML/blob/main/CreateCustomDataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install tesseract
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install shapely
!pip install scikit-learn

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:

import os
import json
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

from shapely.geometry import Polygon
import glob
from PIL import Image
from pytesseract import pytesseract
from lxml import etree
import ast

from sklearn.model_selection import train_test_split


In [3]:
# Open label-studio output json.

f = open('/content/project-4.json')
label_studio_data = json.load(f)

In [19]:
def calculate_iou(box_1, box_2):
    poly_1 = Polygon(box_1)
    poly_2 = Polygon(box_2)
    # print(poly_1,poly_2)
    # iou = poly_1.intersection(poly_2).area / poly_1.union(poly_2).area
    iou = poly_1.intersection(poly_2).area
    min_area = min(poly_1.area,poly_2.area)
    return iou/min_area


def hocr_to_dataframe(fp):
    doc = etree.parse(fp)
    words = []
    wordConf = []
    coords_list = []
    for path in doc.xpath('//*'):
        if 'ocrx_word' in path.values():
            coord_text = path.values()[2].split(';')[0].split(' ')[1:]
            word_coord = list(map(int, coord_text)) #x1, y1, x2, y2
            conf = [x for x in path.values() if 'x_wconf' in x][0]
            wordConf.append(int(conf.split('x_wconf ')[1]))
            words.append(path.text)
            coords_list.append(word_coord)

    dfReturn = pd.DataFrame({'word' : words,
                             'coords': coords_list,
                             'confidence' : wordConf})

    return(dfReturn)

In [20]:
print(label_studio_data)

[{'id': 7, 'annotations': [{'id': 4, 'completed_by': 1, 'result': [{'original_width': 1955, 'original_height': 2530, 'image_rotation': 0, 'value': {'x': 61.687979539641944, 'y': 7.075098814229249, 'width': 25.012787723785166, 'height': 1.0671936758893281, 'rotation': 0}, 'id': '98c80b98-0', 'from_name': 'bbox', 'to_name': 'image', 'type': 'rectangle', 'origin': 'prediction-changed'}, {'original_width': 1955, 'original_height': 2530, 'image_rotation': 0, 'value': {'x': 61.687979539641944, 'y': 7.075098814229249, 'width': 25.012787723785166, 'height': 1.0671936758893281, 'rotation': 0, 'text': ['SEQUENCE SG BIZCO PTE. LTD. (202346260R)']}, 'id': '98c80b98-0', 'from_name': 'transcription', 'to_name': 'image', 'type': 'textarea', 'origin': 'prediction-changed', 'score': 0.5}, {'original_width': 1955, 'original_height': 2530, 'image_rotation': 0, 'value': {'x': 61.687979539641944, 'y': 7.075098814229249, 'width': 25.012787723785166, 'height': 1.0671936758893281, 'rotation': 0, 'labels': ['V

In [21]:
document_data = dict()
document_data['file_name'] = []
document_data['labelled_bbox']= []

for i in range(len(label_studio_data)):
    row = label_studio_data[i]
    file_name = os.path.basename(row['data']['ocr'])
    label_list, labels, bboxes = [], [], []
    for label_ in row['annotations'][0]['result']:
        label_value = label_['value']
        x, y, w, h = label_value['x'], label_value['y'], label_value['width'], label_value['height']
        original_w , original_h = label_['original_width'], label_['original_height']

        x1 = int((x * original_w) / 100)
        y1 = int((y * original_h) / 100)
        x2 = x1 + int(original_w*w / 100)
        y2 = y1 + int(original_h*h / 100)
        try:
           label = label_value['labels']
           print(label)
           label_list.append((label, (x1,y1,x2,y2), original_h, original_w))
        except KeyError:
           var1=1

    document_data['file_name'].append(file_name)
    document_data['labelled_bbox'].append(label_list)

custom_dataset = pd.DataFrame(document_data)

['VENDOR NAME']
['SHIP TO NAME']
['INVOICE NUMBER']
['INVOICE DATE']
['PONUMBER']
['INVOICE DESCRIPTION']
['ITEM']
['UOM']
['QUANTITY']
['LINETOTAL']
['ITEM']
['UOM']
['QUANTITY']
['LINETOTAL']
['ITEM']
['UOM']
['QUANTITY']
['LINETOTAL']
['ITEM']
['UOM']
['QUANTITY']
['LINETOTAL']
['ITEM']
['UOM']
['QUANTITY']
['LINETOTAL']
['ITEM']
['UOM']
['QUANTITY']
['LINETOTAL']
['INVOICE TOTAL']
['TYPE OF DOCUMENT']
['VENDOR NAME']
['TYPE OF DOCUMENT']
['INVOICE NUMBER']
['INVOICE DESCRIPTION']
['PONUMBER']
['INVOICE DATE']
['PO DATE']
['SHIP TO NAME']
['ITEM']
['UOM']
['QUANTITY']
['LINETOTAL']
['TAX AMOUNT']
['INVOICE TOTAL']
['VENDOR NAME']
['TYPE OF DOCUMENT']
['INVOICE NUMBER']
['INVOICE DESCRIPTION']
['INVOICE DATE']
['SHIP TO NAME']
['ITEM']
['UOM']
['QUANTITY']
['LINETOTAL']
['ITEM']
['TAX AMOUNT']
['INVOICE TOTAL']
['VENDOR NAME']
['TYPE OF DOCUMENT']
['INVOICE NUMBER']
['INVOICE DESCRIPTION']
['INVOICE DATE']
['SHIP TO NAME']
['ITEM']
['UOM']
['QUANTITY']
['LINETOTAL']
['INVOICE TOTAL']

In [22]:
print(label_list)

[(['TYPE OF DOCUMENT'], (76, 330, 319, 385), 2200, 1700), (['VENDOR NAME'], (1310, 321, 1591, 351), 2200, 1700), (['INVOICE DATE'], (1002, 346, 1146, 378), 2200, 1700), (['SHIP TO NAME'], (156, 422, 406, 445), 2200, 1700), (['PONUMBER'], (1002, 509, 1106, 534), 2200, 1700), (['ITEM'], (83, 967, 834, 997), 2200, 1700), (['QUANTITY'], (1065, 965, 1125, 999), 2200, 1700), (['LINETOTAL'], (1485, 965, 1573, 997), 2200, 1700), (['ITEM'], (85, 1034, 933, 1056), 2200, 1700), (['QUANTITY'], (1062, 1029, 1125, 1063), 2200, 1700), (['LINETOTAL'], (1499, 1029, 1572, 1061), 2200, 1700), (['ITEM'], (85, 1098, 832, 1121), 2200, 1700), (['QUANTITY'], (1067, 1095, 1123, 1123), 2200, 1700), (['LINETOTAL'], (1485, 1093, 1573, 1125), 2200, 1700), (['ITEM'], (83, 1157, 935, 1187), 2200, 1700), (['QUANTITY'], (1065, 1155, 1125, 1189), 2200, 1700), (['LINETOTAL'], (1499, 1157, 1572, 1189), 2200, 1700), (['ITEM'], (85, 1221, 834, 1251), 2200, 1700), (['QUANTITY'], (1065, 1219, 1125, 1254), 2200, 1700), (['LIN

In [24]:
print(custom_dataset)

                                           file_name  \
0  20240316 201713542C Livinwall - Inv 0001_image...   
1            23 SUZLON ENERGY LIMITED 21_image_0.png   
2                      2324009 NAKODA 15_image_0.png   
3  2324023 Alliancetech International DMCC 18_ima...   
4                             INV-001042_image_0.png   
5                             INV-001043_image_0.png   
6                             INV-001049_image_0.png   
7                           Invoice INV - 0001_1.png   
8                             Invoice INV-0005_1.png   
9                             Quote QU0024 (1)_1.png   

                                       labelled_bbox  
0  [([VENDOR NAME], (1206, 179, 1695, 206), 2530,...  
1  [([VENDOR NAME], (648, 245, 1363, 282), 2530, ...  
2  [([VENDOR NAME], (644, 213, 1309, 250), 2530, ...  
3  [([VENDOR NAME], (653, 245, 1344, 279), 2530, ...  
4  [([VENDOR NAME], (1252, 185, 1778, 241), 2689,...  
5  [([VENDOR NAME], (1252, 185, 1778, 241), 2689,... 

In [30]:
label2id = {"TYPE OF DOCUMENT": 0, "VENDOR NAME": 1, "INVOICE DATE": 2,"SHIP TO NAME": 3,"PONUMBER": 4,"ITEM": 5,"QUANTITY": 6,"LINETOTAL": 7,"INVOICE TOTAL": 8,"INVOICE NUMBER": 9,"INVOICE DESCRIPTION": 10,"TAX AMOUNT": 11,"UOM": 12,"PO DATE": 13}
id2label = {v:k for k, v in label2id.items()}

In [31]:
%%time

final_list = []

for i in tqdm(custom_dataset.iterrows(), total=custom_dataset.shape[0]):
    custom_label_text = {}
    word_list = []
    ner_tags_list  = []
    bboxes_list = []

    file_name = i[1]['file_name']

    for image in glob.glob('/content/layoutlmv3/*.png'): #Make sure you add your extension or change it based on your needs
        frame_file_name = os.path.basename(image)
        if frame_file_name == file_name:
            custom_label_text['id'] = i[0]
            image_basename = os.path.basename(image)
            custom_label_text['file_name'] = image_basename
            annotations = []
            label_coord_list = i[1]['labelled_bbox']
            for label_coord in label_coord_list:
                (x1,y1,x2,y2) = label_coord[1]
                box1 = [[x1, y1], [x2, y1], [x2, y2], [x1, y2]]
                label = label_coord[0][0]
                base_name = os.path.join('/content', 'layoutlmv3',os.path.basename(image).split('.')[0])
                pytesseract.run_tesseract(image, base_name, extension='box', lang=None, config="hocr")
                hocr_file = os.path.join(base_name+'.hocr')
                hocr_df = hocr_to_dataframe(hocr_file)
                for word in hocr_df.iterrows():
                    coords = word[1]['coords']
                    (x1df,y1df,x2df,y2df) = coords
                    box2 = [[x1df, y1df], [x2df, y1df], [x2df, y2df], [x1df, y2df]]
                    words = word[1]['word']
                    overlap_perc = calculate_iou(box1,box2)
                    temp_dic = {}
                    if overlap_perc > 0.80:
                        if words != '-':
                            word_list.append(words)
                            bboxes_list.append(coords)
                            label_id = label2id[label]
                            ner_tags_list.append(label_id)

                        custom_label_text['tokens'] = word_list
                        custom_label_text['bboxes'] = bboxes_list
                        custom_label_text['ner_tags'] = ner_tags_list

    final_list.append(custom_label_text)

100%|██████████| 10/10 [13:31<00:00, 81.18s/it]

CPU times: user 13.7 s, sys: 492 ms, total: 14.2 s
Wall time: 13min 31s


In [32]:
train, test = train_test_split(final_list, random_state=21, test_size=0.3)

for detail  in final_list:
    with open('final_list_text.txt', 'a') as f:
        f.write(str(detail))
        f.write('\n')

for detail  in train:
    with open('train.txt', 'a') as f:
        f.write(str(detail))
        f.write('\n')

for detail  in test:
    with open('test.txt', 'a') as f:
        f.write(str(detail))
        f.write('\n')

In [34]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00


In [35]:
import json
import os
import ast
from pathlib import Path
import datasets
from PIL import Image
import pandas as pd

logger = datasets.logging.get_logger(__name__)
_CITATION = """\
@article{,
  title={},
  author={},
  journal={},
  year={},
  volume={}
}
"""
_DESCRIPTION = """\
This is a sample dataset for training layoutlmv3 model on custom annotated data.
"""

def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    w, h = image.size
    return image, (w,h)

def normalize_bbox(bbox, size):
    return [
        int(1000 * bbox[0] / size[0]),
        int(1000 * bbox[1] / size[1]),
        int(1000 * bbox[2] / size[0]),
        int(1000 * bbox[3] / size[1]),
    ]


_URLS = []

'''Edit your working directory folder path here if required.
If this file is in the same folder as the "layoutlmv3" folder keep it as it is.
'''
data_path = r'./'

class DatasetConfig(datasets.BuilderConfig):
    """BuilderConfig for InvoiceExtraction Dataset"""
    def __init__(self, **kwargs):
        """BuilderConfig for InvoiceExtraction Dataset.
        Args:
          **kwargs: keyword arguments forwarded to super.
        """
        super(DatasetConfig, self).__init__(**kwargs)


class InvoiceExtraction(datasets.GeneratorBasedBuilder):
    BUILDER_CONFIGS = [
        DatasetConfig(name="InvoiceExtraction", version=datasets.Version("1.0.0"), description="InvoiceExtraction dataset"),
    ]

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "tokens": datasets.Sequence(datasets.Value("string")),
                    "bboxes": datasets.Sequence(datasets.Sequence(datasets.Value("int64"))),
                    "ner_tags": datasets.Sequence(
                        datasets.features.ClassLabel(
                            names = ['invoice_no', 'date', 'amount'] #Enter the list of labels that you have here.
                            )
                    ),
                    "image_path": datasets.Value("string"),
                    "image": datasets.features.Image()
                }
            ),
            supervised_keys=None,
            citation=_CITATION,
            homepage="",
        )




    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        """Uses local files located with data_dir"""
        dest = os.path.join(data_path, 'layoutlmv3')

        return [
            datasets.SplitGenerator(
                name=datasets.Split.TRAIN, gen_kwargs={"filepath": os.path.join(dest, "train.txt"), "dest": dest}
            ),
            datasets.SplitGenerator(
                name=datasets.Split.TEST, gen_kwargs={"filepath": os.path.join(dest, "test.txt"), "dest": dest}
            ),
        ]

    def _generate_examples(self, filepath, dest):

        df = pd.read_csv(os.path.join(dest, 'class_list.txt'), delimiter='\s', header=None)
        id2labels = dict(zip(df[0].tolist(), df[1].tolist()))


        logger.info("⏳ Generating examples from = %s", filepath)

        item_list = []
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                item_list.append(line.rstrip('\n\r'))
        print(item_list)
        for guid, fname in enumerate(item_list):
            print(fname)
            data = ast.literal_eval(fname)
            image_path = os.path.join(dest, data['file_name'])
            image, size = load_image(image_path)
            boxes = data['bboxes']

            text = data['tokens']
            label = data['ner_tags']

            #print(boxes)
            #for i in boxes:
            #  print(i)
            boxes = [normalize_bbox(box, size) for box in boxes]
            flag=0
            #print(image_path)
            for i in boxes:
              #print(i)
              for j in i:
                if j>1000:
                  flag+=1
                  #print(j)
                  pass
            if flag>0: print(image_path)

            yield guid, {"id": str(guid), "tokens": text, "bboxes": boxes, "ner_tags": label, "image_path": image_path, "image": image}
